# Simple Alfalfa Example

## Setup
1. [Alfalfa stack deployed](https://github.com/NREL/alfalfa/wiki/Deployment) locally with **at least two workers**
1. [alfalfa](https://github.com/NREL/alfalfa) cloned in same root directory and this alfalfa-notebooks repo.  This is used only for local paths to models, so you can also update those paths and populate models another way.

## Notes
1. For Alfalfa v0.2.0, API reads and writes are implemented over the [Haystack API](https://project-haystack.org/doc/docHaystack/HttpApi), particularly the `read` and `pointWrite` endpoints.


In [ ]:
import os
import datetime
import time
from alfalfa_client.alfalfa_client import AlfalfaClient

In [ ]:
# Create a new client
ac = AlfalfaClient(url='http://localhost')

In [ ]:
# Local paths to models for upload.  Assumes alfalfa and alfalfa-notebooks are cloned in same root dir.

m1 = '../alfalfa/tests/integration/models/single_zone_vav.fmu'
m2 = '../alfalfa/tests/integration/models/refrig_case_osw 2.zip'

In [ ]:
# Upload to Alfalfa 
# 12/10/2021: you will see the models appear in UI at http://localhost/sites
# The metadata is generated on upload and can be viewed at `http://localhost/api/read?filter=point` and `http://localhost/api/read?filter=site`.
# A simulation is a "site".
m1_id = ac.submit(m1)
m2_id = ac.submit(m2)


In [ ]:
print(m1_id)
print(m2_id)

In [ ]:
# Define the run parameters.  
# If you are using historian, you will need to search for this time period in Grafana dashboard to view results.
start_dt = datetime.datetime(2021, 7, 1, 12, 2, 0)
end_dt = datetime.datetime(2021, 7, 3, 0, 0, 0)

# For external_clock == true, API calls are used to advance the model.  
# If external_clock == false, Alfalfa will handle advancing the model according to a specified timescale (timescale 1 => realtime)
params = {
    "external_clock": "true",
    "start_datetime": start_dt,
    "end_datetime": end_dt
}

In [ ]:
# Start simulations 
# Note: one sim runs / worker, so if you have not scaled your local deployment to workers >= 2, 
# the second worker won't have a chance to start and this code block will not complete.

ac.start(m1_id, **params)
ac.start(m2_id, **params)

In [ ]:
# Advance the model.  
# 12/10/2021: timestep is hardcoded to 1 minute w/in Alfalfa worker.
# Model values are exposed as the `curVal` for points and can be viewed at `http://localhost/api/read?filter=point`
ac.advance([m1_id, m2_id])

In [ ]:
# Stop the models
ac.stop(m1_id)
ac.stop(m2_id)